Twilight Struggle is one of the all-time great boardgames. It is a card game where two players compete as opposing Superpower (USSR and US) for Cold War victory.

Card

Each card has Operation Points (Ops) = 1, 2, 3, or 4.

Realignment

Realignment rolls are used to reduce enemy Influence in a country. To attempt a Realignment roll, the acting player need not have any Influence in the target country or in any adjacent country—although this improves the chance of success greatly.

When using a card for Realignment rolls, the player may resolve each roll before declaring the next target.

It costs one Op to make a Realignment roll. Each player rolls a die.

Before rolling, each player has a modifier: +1 for each adjacent controlled country; +1 if they have more Influence in the target country than their opponent; +1 if your Superpower is adjacent to the target country.

Rolled number plus modifier is the final results. High roller wins.

Let **x** be the total modifier calculated from player I minus player II. For this question we will consider x = 0, ±1, ±2, ±3, ±4.

Let **y** be the difference of final results between player I and player II(y ≥ 0). If y > 0, the high roller removes y markers from their opponent’s Influence in the target country. Remove all markers if there are not enough.

At most, the same country can be targeted z times per Action Round, z being the Ops on the card played.


Simulation

This simulation will cover situations for x **starting from** 0, ±1, ±2, ±3, ±4; y is **at least** 1, 2, 3, 4, 5; z = 1.

Notice that, when we attempt to reaglin multiple times in the same country, each attempt affects Influence, which in turn may affect the modifier for the next attempt. For example, player I loses Influence and thus loses the +1 modifier for having more Influence in the target country than player II. In other words, the problem is overcomplicated when we realign multiple times, as it depends on the prior Influence of both Superpowers in target country.

Therefore, we only simulate one realignment attempt, z = 1. Which makes sense because when you are making multiple realignment attempts, you will not make the bonehead decision to spend all your Ops in the same country. Instead, you will decide which country to target for each attempt, based on the result of the previous attempt.

In [1]:
import numpy as np
import pandas as pd

In [2]:
rng = np.random.default_rng()
for i in range(10):
    r1 = rng.integers(1, 7)
    r2 = rng.integers(1, 7)
    print([r1, r2])

[6, 3]
[1, 4]
[5, 6]
[2, 6]
[5, 6]
[5, 1]
[3, 6]
[1, 6]
[3, 3]
[4, 5]


In [3]:
modifiers = [x for x in range(-4, 0)] + [x for x in range(5)]
modifiers

[-4, -3, -2, -1, 0, 1, 2, 3, 4]

Set up one dataframe for each starting modifier to store results (probabilities)

In [4]:
def initialize_probability():
    probability = pd.DataFrame(columns = ['Result'] + ["Modifier " + str(x) for x in modifiers])
    for y in range(1, 6):
        probability.loc[len(probability.index)] = [y] + [0 for x in modifiers]
    probability = probability.set_index(probability['Result'])
    probability = probability.drop(columns='Result')
    return probability

In the dataframe

Result = y: acting player wins by **at least** y from one realignment attempt

The rest of the columns: when acting player has modifier 0, ±1, ±2, ±3, ±4.

The number of simulation runs

In [5]:
n = 10000
probability = initialize_probability()
probability

,Modifier -4,Modifier -3,Modifier -2,Modifier -1,Modifier 0,Modifier 1,Modifier 2,Modifier 3,Modifier 4
Result,,,,,,,,,
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0


In [6]:
for i in range(n):
    for x in modifiers:
        r1 = rng.integers(1, 7)
        r2 = rng.integers(1, 7)
        Result = r1 - r2 + x
        #print('r1 = {}, r2 = {}, x = {}, Result = {}'.format(r1, r2, x, Result))
        for y in range(1, min(Result + 1, 6)):
            #print('r1 = {}, r2 = {}, x = {}, Result = {}, y = {}'.format(r1, r2, x, Result, y))
            probability.at[y, "Modifier " + str(x)] += 1 / n

In [7]:
probability

,Modifier -4,Modifier -3,Modifier -2,Modifier -1,Modifier 0,Modifier 1,Modifier 2,Modifier 3,Modifier 4
Result,,,,,,,,,
1,0.0295,0.0798,0.161,0.2808,0.4181,0.582,0.7237,0.8234,0.9181
2,0,0.0279,0.0824,0.1717,0.2788,0.4166,0.5819,0.7153,0.8362
3,0,0,0.0269,0.0822,0.166,0.2718,0.4193,0.5786,0.7246
4,0,0,0,0.0272,0.0811,0.1632,0.275,0.4044,0.5832
5,0,0,0,0,0.0273,0.0824,0.1667,0.2656,0.4164
